# SelfQueryRetriever - 지능형 검색 시스템

## 개요

**SelfQueryRetriever**는 자연어 질의를 구조화된 검색 조건으로 변환하여 메타데이터 기반의 정밀한 검색을 수행하는 고급 검색 도구입니다.

## 일반 검색 vs SelfQueryRetriever 비교

### 일반 검색의 한계

일반적인 키워드 기반 검색 시스템:
- 사용자: "2023년도 스킨케어 제품 추천해줘"
- 시스템: "2023", "스킨케어", "추천" 키워드로만 단순 검색
- **문제점**: 연도와 카테고리 조건을 구분하지 못함

### SelfQueryRetriever의 지능형 검색

동일한 질의에 대한 SelfQueryRetriever 처리:
- 사용자: "2023년도 스킨케어 제품 추천해줘"
- 시스템 분석:
  - 연도 조건: `year = 2023`
  - 카테고리 조건: `category = "스킨케어"`
- **결과**: 메타데이터 필터를 적용하여 정확한 검색 수행

## SelfQueryRetriever의 핵심 장점

| 특징 | 설명 | 장점 |
|------|------|------|
| **정밀 필터링** | 메타데이터를 활용한 구조화된 검색 | 검색 정확도 대폭 향상 |
| **자연어 처리** | 복잡한 조건을 자연어로 표현 가능 | 사용자 편의성 증대 |
| **유연한 조건** | 다양한 비교 연산자 지원 | 세밀한 검색 조건 설정 |
| **의도 파악** | 사용자 질의 의도를 정확하게 해석 | 검색 만족도 향상 |

## 학습 목표

이 튜토리얼을 통해 다음 내용을 학습합니다:

1. **환경 설정**: 튜토리얼 실행 환경 구성
2. **샘플 데이터 생성**: 화장품 데이터를 활용한 실습 환경 구축
3. **SelfQueryRetriever 구성**: 핵심 구성 요소 설정
4. **기본 검색**: 단일 조건 검색 테스트
5. **복합 조건 검색**: 다중 조건 동시 적용
6. **검색 결과 제한**: k 파라미터를 통한 결과 수 조절
7. **내부 구조 분석**: Query Constructor와 Translator의 역할 이해

## 핵심 개념

- **Query Constructor**: 자연어 질의를 구조화된 쿼리로 변환
- **Structured Query Translator**: 구조화된 쿼리를 벡터스토어 전용 필터로 변환
- **메타데이터 필터링**: 속성 기반의 정밀한 검색 조건 적용
- **AttributeInfo**: 메타데이터 필드의 타입과 설명 정보 정의

In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv(override=True)

## 환경 설정

SelfQueryRetriever 사용을 위한 필수 라이브러리와 API 키 설정을 진행합니다.

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangChain-Tutorial")

## 샘플 데이터 생성

### 화장품 데이터베이스 구축

실제 화장품 온라인몰에서 사용할 수 있는 상품 데이터를 생성하여 SelfQueryRetriever의 기능을 실습합니다.

### 데이터 구조

각 화장품 제품은 다음과 같은 정보를 포함합니다:

| 속성 | 데이터 타입 | 설명 | 예시 |
|------|-------------|------|------|
| **page_content** | string | 제품 상세 설명 | "수분 가득한 히알루론산 세럼..." |
| **year** | integer | 제품 출시 연도 | 2023, 2024 |
| **category** | string | 제품 카테고리 | 스킨케어, 메이크업, 클렌징, 선케어 |
| **user_rating** | float | 사용자 평점 (1-5점) | 4.7, 4.5, 4.8 |

### 벡터 저장소 구축

화장품 상품 정보를 **벡터 형태로 저장**하여 의미적 유사도 검색이 가능한 데이터베이스를 구축합니다.

#### 벡터 저장소의 역할

벡터 저장소는 텍스트를 수치 벡터로 변환하여 저장하는 데이터베이스입니다. 이를 통해:

- **의미적 유사도 검색**: 키워드 일치가 아닌 의미적 유사성으로 검색
- **메타데이터 필터링**: 구조화된 속성 정보를 활용한 정밀 검색
- **하이브리드 검색**: 벡터 검색과 메타데이터 필터링의 결합

In [ ]:
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

# 화장품 상품의 설명과 메타데이터 생성
docs = [
    Document(
        page_content="수분 가득한 히알루론산 세럼으로 피부 속 깊은 곳까지 수분을 공급합니다.",
        metadata={"year": 2024, "category": "스킨케어", "user_rating": 4.7},  # 메타데이터: 연도, 카테고리, 평점
    ),
    Document(
        page_content="24시간 지속되는 매트한 피니시의 파운데이션, 모공을 커버하고 자연스러운 피부 표현이 가능합니다.",
        metadata={"year": 2023, "category": "메이크업", "user_rating": 4.5},  # 메타데이터: 연도, 카테고리, 평점
    ),
    Document(
        page_content="식물성 성분으로 만든 저자극 클렌징 오일, 메이크업과 노폐물을 부드럽게 제거합니다.",
        metadata={"year": 2023, "category": "클렌징", "user_rating": 4.8},  # 메타데이터: 연도, 카테고리, 평점
    ),
    Document(
        page_content="비타민 C 함유 브라이트닝 크림, 칙칙한 피부톤을 환하게 밝혀줍니다.",
        metadata={"year": 2023, "category": "스킨케어", "user_rating": 4.6},  # 메타데이터: 연도, 카테고리, 평점
    ),
    Document(
        page_content="롱래스팅 립스틱, 선명한 발색과 촉촉한 사용감으로 하루종일 편안하게 사용 가능합니다.",
        metadata={"year": 2024, "category": "메이크업", "user_rating": 4.4},  # 메타데이터: 연도, 카테고리, 평점
    ),
    Document(
        page_content="자외선 차단 기능이 있는 톤업 선크림, SPF50+/PA++++ 높은 자외선 차단 지수로 피부를 보호합니다.",
        metadata={"year": 2024, "category": "선케어", "user_rating": 4.9},  # 메타데이터: 연도, 카테고리, 평점
    ),
]

# 벡터 저장소 생성: 문서들을 벡터로 변환하여 Chroma DB에 저장
vectorstore = Chroma.from_documents(
    docs, OpenAIEmbeddings(model="text-embedding-3-small")  # OpenAI 임베딩 모델 사용
)

## SelfQueryRetriever 구성

### 지능형 검색 시스템 구축

SelfQueryRetriever가 올바르게 작동하려면 두 가지 핵심 정보가 필요합니다:

#### 필수 구성 요소

| 구성 요소 | 역할 | 예시 |
|-----------|------|------|
| **메타데이터 필드 정보** | 가능한 필터링 조건 정의 | "카테고리는 문자열, 연도는 숫자, 평점은 실수" |
| **문서 내용 설명** | 저장된 문서의 주제 요약 | "화장품 상품 정보" |

#### 정보 제공의 중요성

이러한 정보가 필요한 이유:

1. **필드 타입 인식**: AI가 각 메타데이터 필드의 데이터 타입을 정확히 파악
2. **검색 범위 이해**: 어떤 종류의 문서를 검색하는지 맥락 제공
3. **쿼리 변환 정확도**: 자연어 질의를 적절한 필터 조건으로 변환
4. **오류 방지**: 잘못된 필터 조건 생성 방지

#### 동작 과정

```
사용자 질의 → 메타데이터 분석 → 필터 조건 생성 → 정밀 검색 → 결과 반환
```

### 메타데이터 필드 정보 정의

`AttributeInfo`는 각 메타데이터 필드의 세부 정보를 정의하는 클래스입니다. 

#### 화장품 데이터 메타데이터 정의

각 필드별 상세 정보:

| 필드명 | 데이터 타입 | 설명 | 가능한 값 |
|--------|-------------|------|-----------|
| **category** | string | 화장품 카테고리 | 스킨케어, 메이크업, 클렌징, 선케어 |
| **year** | integer | 제품 출시 연도 | 2023, 2024 |
| **user_rating** | float | 고객 만족도 점수 | 1.0 ~ 5.0 |

#### AttributeInfo의 역할

AI가 사용자 질의를 해석할 때:

1. **타입 기반 연산**: "평점이 4.5 이상" → `user_rating >= 4.5` (float 비교)
2. **값 범위 검증**: 허용되지 않는 카테고리 값 필터링
3. **적절한 비교 연산자 선택**: 문자열은 등호(=), 숫자는 부등호(>, <, >=, <=) 사용

#### 정의 예시

```python
AttributeInfo(
    name="user_rating",
    description="A user rating for the cosmetic product, ranging from 1 to 5",
    type="float",
)
```

In [ ]:
from langchain.chains.query_constructor.base import AttributeInfo


# 메타데이터 필드 정보 생성: AI가 각 필드를 어떻게 해석해야 하는지 알려주는 정보
metadata_field_info = [
    AttributeInfo(
        name="category",  # 필드명: category
        description="The category of the cosmetic product. One of ['스킨케어', '메이크업', '클렌징', '선케어']",  # 화장품 카테고리 설명
        type="string",  # 데이터 타입: 문자열
    ),
    AttributeInfo(
        name="year",  # 필드명: year
        description="The year the cosmetic product was released",  # 제품 출시 연도 설명
        type="integer",  # 데이터 타입: 정수
    ),
    AttributeInfo(
        name="user_rating",  # 필드명: user_rating
        description="A user rating for the cosmetic product, ranging from 1 to 5",  # 사용자 평점 설명 (1-5점)
        type="float",  # 데이터 타입: 실수
    ),
]

### SelfQueryRetriever 객체 생성

모든 구성 요소를 조합하여 지능형 검색 시스템을 구축합니다.

#### 필요 구성 요소

| 구성 요소 | 역할 | 설명 |
|-----------|------|------|
| **llm** | 언어 모델 | 사용자 질의를 이해하고 분석 |
| **vectorstore** | 벡터 데이터베이스 | 상품 정보가 저장된 검색 대상 |
| **document_contents** | 문서 내용 설명 | 검색 대상 문서의 주제 요약 |
| **metadata_field_info** | 메타데이터 정보 | 필터링 가능한 속성들의 정의 |

#### SelfQueryRetriever 처리 과정

1. **질의 수신**: "2023년도 스킨케어 제품 추천해줘"
2. **질의 분석**: LLM이 자연어 질의를 구조화된 조건으로 변환
3. **필터 생성**: `year=2023 AND category="스킨케어"`
4. **검색 실행**: 벡터스토어에서 조건에 맞는 제품 검색
5. **결과 반환**: 필터링된 검색 결과를 사용자에게 제공

#### 검색 정확도 향상

SelfQueryRetriever는 다음과 같은 방식으로 검색 정확도를 향상시킵니다:

- **의미적 검색**: 텍스트 내용의 의미적 유사도 활용
- **메타데이터 필터링**: 구조화된 속성 조건 적용
- **하이브리드 검색**: 두 방식의 결합으로 최적의 결과 제공

In [ ]:
import os
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI

# LLM 정의: 사용자 질문을 이해하고 구조화된 쿼리를 생성할 언어 모델
llm = ChatOpenAI(
    model="openai/gpt-4.1",
    temperature=0,
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url=os.getenv("OPENROUTER_BASE_URL"),
)

# SelfQueryRetriever 생성: 자연어 질문을 메타데이터 필터가 포함된 검색으로 변환
retriever = SelfQueryRetriever.from_llm(
    llm=llm,  # 사용할 언어 모델
    vectorstore=vectorstore,  # 검색할 벡터 저장소
    document_contents="Brief summary of a cosmetic product",  # 문서 내용에 대한 간단한 설명
    metadata_field_info=metadata_field_info,  # 메타데이터 필드 정보
)

## 기본 검색 테스트

### 단일 조건 검색 실습

SelfQueryRetriever의 지능형 검색 기능을 다양한 단일 조건으로 테스트합니다.

#### 테스트 시나리오

각 검색에서 SelfQueryRetriever가 자동으로 생성하는 필터 조건을 확인합니다:

| 테스트 케이스 | 사용자 질의 | 자동 생성 필터 | 검색 대상 |
|---------------|-------------|----------------|-----------|
| **평점 조건** | "평점이 4.8 이상인 제품" | `user_rating >= 4.8` | 고평점 제품 |
| **연도 조건** | "2023년에 출시된 상품" | `year = 2023` | 특정 연도 제품 |
| **카테고리 조건** | "선케어 상품" | `category = "선케어"` | 특정 카테고리 제품 |

#### 검색 결과 분석 포인트

- **필터 정확도**: 자연어 질의가 올바른 메타데이터 필터로 변환되는가?
- **검색 범위**: 지정된 조건에 맞는 제품만 반환되는가?
- **결과 품질**: 검색된 제품이 사용자 의도에 부합하는가?

In [ ]:
# Self-query 검색: 평점 조건 필터링
# "평점이 4.8 이상인 제품" → user_rating >= 4.8 조건으로 자동 변환
retriever.invoke("평점이 4.8 이상인 제품을 추천해주세요")

In [ ]:
# Self-query 검색: 연도 조건 필터링  
# "2023년에 출시된 상품" → year = 2023 조건으로 자동 변환
retriever.invoke("2023년에 출시된 상품을 추천해주세요")

In [ ]:
# Self-query 검색: 카테고리 조건 필터링
# "카테고리가 선케어인 상품" → category = "선케어" 조건으로 자동 변환
retriever.invoke("카테고리가 선케어인 상품을 추천해주세요")

## 복합 조건 검색

### 다중 조건 동시 적용

SelfQueryRetriever의 진정한 강점은 **복합 조건 검색**에 있습니다.

#### 복합 조건 검색의 장점

| 구분 | 일반 검색 | SelfQueryRetriever |
|------|-----------|-------------------|
| **처리 방식** | "메이크업 4.5점" 키워드 검색 | `category = "메이크업" AND user_rating >= 4.5` |
| **검색 정확도** | 키워드 일치에 의존 | 정확한 조건 필터링 |
| **결과 품질** | 부정확한 결과 포함 가능 | 조건을 만족하는 결과만 반환 |

#### 실제 활용 시나리오

복합 조건 검색은 다음과 같은 상황에서 유용합니다:

- **전자상거래**: "브랜드 + 가격대 + 평점" 동시 필터링
- **부동산 검색**: "지역 + 가격 + 면적" 조건 조합
- **도서 검색**: "장르 + 출간연도 + 평점" 복합 조건
- **여행 검색**: "지역 + 예산 + 평점 + 날짜" 다중 조건

#### 검색 과정

```
사용자 질의: "메이크업 제품 중 평점 4.5점 이상"
↓
조건 분해: category = "메이크업" AND user_rating >= 4.5
↓
필터 적용: 두 조건을 모두 만족하는 제품만 검색
↓
결과 반환: 정확히 일치하는 제품 목록
```

In [ ]:
# Self-query 검색: 복합 조건 필터링 (카테고리 + 평점)
# "메이크업 + 평점 4.5 이상" → category = "메이크업" AND user_rating >= 4.5 조건으로 자동 변환
retriever.invoke(
    "카테고리가 메이크업인 상품 중에서 평점이 4.5 이상인 상품을 추천해주세요"
)

## 검색 결과 수 제한

### k 파라미터를 통한 결과 제어

검색 결과의 개수를 제한하여 성능 최적화와 사용자 경험을 향상시킬 수 있습니다.

#### k 파라미터 개념

| 설정 방법 | 구현 방식 | 예시 |
|-----------|-----------|------|
| **코드 설정** | `search_kwargs={"k": N}` | 항상 N개 결과 반환 |
| **자연어 설정** | 질의에 숫자 포함 | "3개만 추천해줘" → 자동으로 k=3 |

#### 결과 수 제한의 이점

**성능 최적화**
- 네트워크 전송량 감소
- 응답 속도 향상
- 메모리 사용량 절약

**사용자 경험 개선**
- 핵심 결과에 집중
- 화면 공간 효율적 활용
- 빠른 의사결정 지원

#### 적용 시나리오

| 상황 | 권장 k 값 | 이유 |
|------|-----------|------|
| **빠른 미리보기** | k=2~3 | 대략적인 결과 확인 |
| **상세 비교** | k=5~10 | 충분한 선택지 제공 |
| **모바일 화면** | k=3~5 | 화면 크기 제약 |
| **API 응답 최적화** | k=1~3 | 응답 속도 우선 |

In [ ]:
# 검색 결과 수 제한 기능이 있는 retriever 생성
retriever = SelfQueryRetriever.from_llm(
    llm=llm,  # 언어 모델
    vectorstore=vectorstore,  # 벡터 저장소
    document_contents="Brief summary of a cosmetic product",  # 문서 내용 설명
    metadata_field_info=metadata_field_info,  # 메타데이터 필드 정보
    enable_limit=True,  # 검색 결과 제한 기능을 활성화
    search_kwargs={"k": 2},  # k 값을 2로 지정하여 검색 결과를 2개로 제한
)

#### 실험 환경

테스트 데이터에는 **2023년 출시 상품이 3개**가 있지만, `k=2` 설정으로 **상위 2개만** 반환하여 결과 제한 기능을 확인합니다.

In [ ]:
# Self-query 검색: k=2로 제한된 결과
# 2023년 출시 상품은 3개 있지만 k=2 설정으로 상위 2개만 반환
retriever.invoke("2023년에 출시된 상품을 추천해주세요")

### 자연어를 통한 결과 수 지정

**직관적인 방법**으로 `search_kwargs` 설정 없이도 질의 내에 숫자를 포함시켜 AI가 자동으로 결과 수를 조절하도록 할 수 있습니다.

#### 자연어 개수 지정의 장점

| 방식 | 사용성 | 유연성 | 실제 서비스 적용 |
|------|--------|--------|------------------|
| **코드 설정** | 개발자 친화적 | 고정된 개수 | 시스템 레벨 제어 |
| **자연어 지정** | 사용자 친화적 | 동적 개수 조절 | 대화형 인터페이스 |

#### 자연어 패턴 예시

- **"1개만 추천해줘"** → k=1 자동 설정
- **"상위 3개 보여줘"** → k=3 자동 설정  
- **"5개 정도만"** → k=5 자동 설정
- **"몇 개 추천"** → 기본값 사용

이러한 방식은 사용자가 **더 자연스럽게** 요청할 수 있어서 실제 서비스 환경에서 매우 유용합니다.

In [ ]:
# 자연어로 개수 제한이 가능한 retriever 생성 (search_kwargs 없음)
retriever = SelfQueryRetriever.from_llm(
    llm=llm,  # 언어 모델
    vectorstore=vectorstore,  # 벡터 저장소  
    document_contents="Brief summary of a cosmetic product",  # 문서 내용 설명
    metadata_field_info=metadata_field_info,  # 메타데이터 필드 정보
    enable_limit=True,  # 검색 결과 제한 기능을 활성화 (자연어 개수 인식 가능)
)

# Self-query 검색: 자연어로 개수 지정
# "1개를 추천해주세요" → AI가 자동으로 k=1 설정
retriever.invoke("2023년에 출시된 상품 1개를 추천해주세요")

In [ ]:
# Self-query 검색: 자연어로 개수 지정 (2개)
# "2개를 추천해주세요" → AI가 자동으로 k=2 설정
retriever.invoke("2023년에 출시된 상품 2개를 추천해주세요")

## 내부 동작 원리 분석

### SelfQueryRetriever 아키텍처

SelfQueryRetriever의 내부 구조를 분석하여 동작 메커니즘을 이해합니다.

#### 핵심 구성 요소

SelfQueryRetriever는 두 개의 주요 컴포넌트로 구성됩니다:

| 컴포넌트 | 역할 | 입력 | 출력 |
|----------|------|------|------|
| **Query Constructor** | 쿼리 생성기 | 자연어 질의 | 구조화된 쿼리 |
| **Structured Query Translator** | 쿼리 변환기 | 구조화된 쿼리 | 벡터스토어 전용 필터 |

#### 처리 흐름

```
자연어 질의 → Query Constructor → 구조화된 쿼리 → Query Translator → 벡터스토어 필터 → 검색 실행
```

#### 아키텍처 분리의 장점

**모듈화**
- 각 컴포넌트의 독립적 개발 및 테스트
- 기능별 최적화 가능
- 유지보수 편의성 향상

**확장성**
- 다양한 벡터스토어 지원
- Query Constructor 재사용
- 새로운 Translator 추가 용이

#### 직접 구현의 이점

`SelfQueryRetriever.from_llm()` 대신 직접 조립하는 경우:

| 항목 | 장점 | 활용 사례 |
|------|------|-----------|
| **프롬프트 커스터마이징** | 도메인 특화 최적화 | 전문 용어 처리 개선 |
| **에러 핸들링** | 예외 상황 대응 | 잘못된 질의 처리 |
| **성능 최적화** | 불필요한 단계 제거 | 응답 속도 향상 |
| **디버깅** | 단계별 결과 확인 | 문제점 파악 및 해결 |

#### 참고 자료

- [LangChain 공식 쿡북](https://github.com/langchain-ai/langchain/blob/master/cookbook/self_query_hotel_search.ipynb)
- Query Constructor 고급 설정 방법
- 다양한 벡터스토어 Translator 사용법

### Query Constructor Chain 생성

**Query Constructor**는 SelfQueryRetriever의 핵심 두뇌로서 자연어 질의를 구조화된 검색 조건으로 변환합니다.

#### 구성 요소

| 구성 요소 | 역할 | 설명 |
|-----------|------|------|
| **Prompt** | AI 지시서 | 메타데이터 정보와 예시를 포함한 변환 가이드 |
| **Output Parser** | 응답 파서 | AI 응답을 Python 객체로 변환 |
| **LLM** | 언어 모델 | 실제 질의 이해 및 분석 수행 |

#### Query Constructor의 역할

```python
# 입력 예시
user_query = "2023년 스킨케어 제품 중 평점 4.5 이상"

# 출력 예시
structured_query = {
    "query": "스킨케어 제품",
    "filter": {
        "and": [
            {"year": {"eq": 2023}},
            {"category": {"eq": "스킨케어"}},
            {"user_rating": {"gte": 4.5}}
        ]
    }
}
```

#### 프롬프트 생성 과정

`get_query_constructor_prompt()` 함수는 다음 정보를 조합하여 AI용 지시서를 생성합니다:

- **메타데이터 필드 설명**: 각 필드의 타입과 가능한 값
- **문서 내용 요약**: 검색 대상 문서의 특성
- **변환 규칙**: 자연어를 구조화된 쿼리로 변환하는 방법
- **예시**: 성공적인 변환 사례들

### 쿼리 생성기 프롬프트 구조

`get_query_constructor_prompt()` 함수는 AI가 자연어 질의를 정확하게 해석할 수 있도록 하는 **종합적인 가이드**를 생성합니다.

#### 프롬프트 구성 요소

| 구성 요소 | 내용 | 목적 |
|-----------|------|------|
| **메타데이터 필드 설명** | 필터링 가능한 속성 정보 | AI의 필드 이해도 향상 |
| **문서 내용 요약** | 검색 대상 문서의 주제 | 검색 맥락 제공 |
| **변환 예시** | 성공적인 질의 변환 사례 | 학습 패턴 제공 |
| **출력 형식** | 결과 구조화 방법 | 일관된 응답 형식 보장 |

#### 프롬프트의 중요성

효과적인 프롬프트는 다음과 같은 역할을 수행합니다:

1. **명확한 지시사항**: "이런 질문이 들어오면 이렇게 해석하라"
2. **컨텍스트 제공**: "우리는 화장품 데이터를 다루고 있다"
3. **오류 방지**: "이런 실수를 하지 마라"
4. **일관성 보장**: "결과는 항상 이 형식으로 제공하라"

#### 실무 최적화 팁

프롬프트 품질 향상을 위한 방법:

- **도메인 특화 예시 추가**: 실제 사용 케이스 반영
- **에지 케이스 처리**: 예외 상황 대응 방법 명시
- **명확한 필드 설명**: 데이터 타입과 가능한 값 범위 구체화

In [ ]:
from langchain.chains.query_constructor.base import (
    StructuredQueryOutputParser,
    get_query_constructor_prompt,
)

# 문서 내용 설명과 메타데이터 필드 정보를 사용하여 쿼리 생성기 프롬프트를 가져옵니다.
prompt = get_query_constructor_prompt(
    "Brief summary of a cosmetic product",  # 문서 내용 설명
    metadata_field_info,  # 메타데이터 필드 정보 (위에서 정의한 AttributeInfo 리스트)
)

# StructuredQueryOutputParser 를 생성: AI 응답을 구조화된 쿼리 객체로 변환
output_parser = StructuredQueryOutputParser.from_components()

# query_constructor chain 을 생성: prompt → llm → output_parser 순으로 연결
query_constructor = prompt | llm | output_parser

#### 생성된 프롬프트 내용 확인

AI에게 전달되는 실제 지시사항을 확인하여 프롬프트의 구조와 내용을 이해합니다.

이를 통해 다음을 파악할 수 있습니다:

- **메타데이터 필드 정의**: AI가 각 필드를 어떻게 인식하는지
- **변환 규칙**: 자연어를 구조화된 쿼리로 변환하는 방법
- **출력 형식**: 응답해야 할 JSON 구조
- **예시 패턴**: 성공적인 변환 사례들

In [ ]:
# 프롬프트 내용 출력: AI에게 어떤 지시사항을 주고 있는지 확인
# "dummy question"을 매개변수로 하여 실제 프롬프트 템플릿을 확인
print(prompt.format(query="dummy question"))

#### Query Constructor 실행 테스트

복잡한 조건이 포함된 자연어 질의를 Query Constructor에 입력하여 구조화된 쿼리로 변환되는 과정을 확인합니다.

##### 테스트 질의

**"2023년도에 출시한 상품 중 평점이 4.5 이상인 상품중에서 스킨케어 제품을 추천해주세요"**

이 질의에는 다음 세 가지 조건이 포함되어 있습니다:

| 조건 유형 | 조건 내용 | 예상 필터 |
|-----------|-----------|-----------|
| **연도 조건** | "2023년도에 출시" | `year = 2023` |
| **평점 조건** | "평점이 4.5 이상" | `user_rating >= 4.5` |
| **카테고리 조건** | "스킨케어 제품" | `category = "스킨케어"` |

AI가 이 복합 조건들을 어떻게 정확한 구조화된 쿼리로 변환하는지 관찰할 수 있습니다.

In [ ]:
# Query Constructor 실행: 복잡한 자연어 질문을 구조화된 쿼리로 변환
query_output = query_constructor.invoke(
    {
        # 복잡한 조건이 포함된 질문: 연도(2023) + 평점(4.5이상) + 카테고리(스킨케어)  
        "query": "2023년도에 출시한 상품 중 평점이 4.5 이상인 상품중에서 스킨케어 제품을 추천해주세요"
    }
)

#### 구조화된 쿼리 결과 분석

Query Constructor가 자연어 질의를 어떻게 해석하고 구조화된 조건으로 변환했는지 확인합니다.

**원본 질의**: "2023년도에 출시한 상품 중 평점이 4.5 이상인 상품중에서 스킨케어 제품을 추천해주세요"

##### 기대되는 변환 결과

AI가 추출해야 할 핵심 필터 조건들:

- **연도 필터**: `year` 필드에 2023 값 적용
- **평점 필터**: `user_rating` 필드에 4.5 이상 조건 적용  
- **카테고리 필터**: `category` 필드에 "스킨케어" 값 적용
- **논리 연산**: 모든 조건을 AND로 결합

In [ ]:
# 생성된 구조화 쿼리의 필터 조건 확인
# AI가 자연어 질문에서 추출한 필터 조건들을 출력
query_output.filter.arguments

### Query Constructor 최적화 전략

**SelfQueryRetriever의 성능은 Query Constructor의 품질에 직접적으로 의존합니다.** 효과적인 최적화를 통해 검색 정확도를 크게 향상시킬 수 있습니다.

#### 최적화 방법론

| 최적화 영역 | 방법 | 효과 |
|-------------|------|------|
| **프롬프트 튜닝** | 도메인 특화 예시 추가 | 변환 정확도 향상 |
| **속성 설명 개선** | 명확한 필드 정의 | 오류 감소 |
| **예시 데이터 확장** | Few-shot 학습 강화 | 복잡한 질의 처리 개선 |

#### 실무 최적화 체크리스트

**프롬프트 품질 개선**
- 도메인 전문 용어 정의 추가
- 자주 발생하는 실수 패턴 방지 규칙 명시
- 성공적인 변환 사례 다양화

**메타데이터 정의 강화**
- 데이터 타입과 범위 명확히 명시
- 가능한 값들의 구체적 예시 제공
- 필드 간 관계성 설명 추가

**성능 검증 프로세스**
- 다양한 질의 패턴으로 반복 테스트
- 실패 사례 분석 및 프롬프트 개선
- A/B 테스트를 통한 최적화 효과 검증

#### 지속적 개선 전략

효과적인 SelfQueryRetriever 구축을 위한 핵심 원칙:

1. **도메인 전문성**: 해당 분야의 용어와 패턴 숙지
2. **사용자 행동 분석**: 실제 질의 패턴 파악 및 반영
3. **반복적 검증**: 지속적인 테스트와 개선
4. **데이터 기반 최적화**: 실패 사례 분석을 통한 체계적 개선

### 구조화된 쿼리 변환기 (Structured Query Translator)

#### 벡터스토어별 전용 변환기

**Structured Query Translator**는 각 벡터스토어의 고유한 쿼리 문법에 맞춰 구조화된 쿼리를 변환하는 컴포넌트입니다.

#### 벡터스토어별 문법 차이

동일한 조건 `{year: 2023}`을 각 벡터스토어에서 표현하는 방법:

| 벡터스토어 | 필터 문법 | 예시 |
|------------|-----------|------|
| **Chroma** | MongoDB 스타일 | `{"year": {"$eq": 2023}}` |
| **Pinecone** | 키-값 형식 | `year=2023` |
| **Weaviate** | GraphQL 스타일 | `where: {path: ["year"], operator: Equal, valueInt: 2023}` |

#### ChromaTranslator의 역할

Chroma 데이터베이스 사용 시 ChromaTranslator가 수행하는 변환:

```python
# 입력: 표준 구조화 쿼리
structured_query = {
    "query": "스킨케어 제품",
    "filter": {"year": {"eq": 2023}}
}

# 출력: Chroma 전용 필터
chroma_filter = {"year": {"$eq": 2023}}
```

#### 아키텍처의 확장성

이러한 분리된 구조의 장점:

| 장점 | 설명 | 활용 |
|------|------|------|
| **벡터스토어 독립성** | Query Constructor 재사용 가능 | 다양한 DB 지원 |
| **유지보수성** | 각 컴포넌트 독립적 수정 | 문법 변경 대응 |
| **확장성** | 새 벡터스토어 지원 용이 | Translator만 추가 |

In [ ]:
from langchain.retrievers.self_query.chroma import ChromaTranslator

# 직접 구성한 구성 요소들로 SelfQueryRetriever 생성
retriever = SelfQueryRetriever(
    query_constructor=query_constructor,  # 이전에 생성한 query_constructor chain을 지정 (자연어 → 구조화 쿼리)
    vectorstore=vectorstore,  # 검색할 벡터 저장소 (Chroma DB)
    structured_query_translator=ChromaTranslator(),  # 쿼리 변환기 (구조화 쿼리 → Chroma 필터 문법)
)

### 완성된 SelfQueryRetriever 테스트

직접 구축한 SelfQueryRetriever의 최종 동작을 확인합니다.

#### 테스트 시나리오

**복합 조건 질의**: "2023년도에 출시한 상품 중 평점이 4.5 이상인 상품중에서 스킨케어 제품을 추천해주세요"

#### 예상 처리 과정

1. **Query Constructor**: 자연어 → 구조화 쿼리 변환
2. **ChromaTranslator**: 구조화 쿼리 → Chroma 필터 변환
3. **Vectorstore**: 필터링된 검색 실행
4. **결과 반환**: 모든 조건을 만족하는 제품들

#### 검증 포인트

- **구성 요소 통합**: 각 컴포넌트가 올바르게 연결되어 작동하는가?
- **결과 정확성**: `from_llm()` 방식과 동일한 결과를 제공하는가?
- **성능**: 직접 구축 방식의 성능은 어떠한가?

In [ ]:
# 직접 구성한 SelfQueryRetriever 최종 테스트
# 복잡한 조건의 질문을 통해 모든 구성 요소가 제대로 작동하는지 확인
retriever.invoke(
    # 질문: 여러 조건이 포함된 복합 질문 (연도 + 평점 + 카테고리)
    "2023년도에 출시한 상품 중 평점이 4.5 이상인 상품중에서 스킨케어 제품을 추천해주세요"
)